In [15]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import nntools as nt
import ctoolbox

import pdb
import collections
from pathlib import Path
import json
from matplotlib import patches, patheffects


In [16]:
from encoder import DataEncoder
import multibox_layer
import multibox_loss

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# 1. Load database

In [3]:
root_dir = '/datasets/ee285f-public/PascalVOC2012/JPEGImages/'

In [10]:
class PascalVocDataset(td.Dataset):
    img_size = 300
    
    def __init__(self, root_dir, mode='train'):
        super(PascalVocDataset, self).__init__()
        self.root_dir = root_dir
        self.mode = mode
        self.dict= json.load( (Path().absolute()/('pascal_%s2012.json'%mode)).open())      
        # id :  bbox/class id  
        annotation=collections.defaultdict(lambda:[])
        for o in self.dict['annotations']:
            if not o['ignore']:
                bb = o['bbox']
                bb = hw_bb(bb)
                annotation[o['image_id']].append((bb,o['category_id']))
        self.anno = annotation
        #class 
        self.class_dict = {o['id']:o['name'] for o in self.dict['categories']}
        self.data_encoder =  DataEncoder()
        
    def __len__(self):
        return len(self.dict['images'])
    
    def __repr__(self):
        return "PascalVocDataset(mode={})".format(self.mode)
    
    def __getitem__(self, idx):
        fname = self.dict['images'][idx]['file_name']
        img_id = self.dict['images'][idx]['id']
        #pdb.set_trace()        
        img_path = os.path.join(self.root_dir, fname)
        x = Image.open(img_path).convert('RGB')        
        anno = self.anno[img_id]     
        boxes = []
        labels = []
        for i in anno:
            boxes.append(i[0])
            labels.append(i[1])
        w,h = x.size
        
        pdb.set_trace()
        boxes_t = torch.Tensor(boxes)
        lables_t = torch.LongTensor(lables)
        
        boxes_t /=torch.Tensor([w,h,w,h]).expand_as(boxes_t)
        lables_t = torch.LongTensor(lables_t)
        
        D_encode =self.data_encoder
        loc_target, conf_target = D_encode.encode(boxes_t,lables_t)
        
        x = x.resize((self.img_size,self.img_size))
        transform = tv.transforms.Compose([
            tv.transforms.ToTensor(),
            tv.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) #imageNet Stats
        ])
        x = transform(x) 
        d = (loc_target,conf_target)

        return x,d
    def getclass(self,idx):
        return self.class_dict[idx]
        

In [12]:
train_set= PascalVocDataset(root_dir)
x, d= train_set[0]
plt.figure(figsize=(5,3))
myimshow(x)

NameError: name 'hw_bb' is not defined

In [6]:
x, d= train_set[0]
draw_image(x)

NameError: name 'train_set' is not defined

## 2. Draw image and bbox 

In [21]:
def myimshow(image, ax=plt):
    '''
    Accept image as tensor
    '''
    image = image.to('cpu').numpy()
    image = np.moveaxis(image, [0, 1, 2], [2, 0, 1])
    image = (image + 1) / 2
    image[image < 0] = 0
    image[image > 1] = 1
    h = ax.imshow(image)
    ax.axis('off')
    return h

In [7]:
train_set= PascalVocDataset(root_dir)
x, d= train_set[0]
plt.figure(figsize=(5,3))
myimshow(x)

NameError: name 'hw_bb' is not defined

In [39]:
d[0]

tensor([[27.5667, 17.5581,     nan,  3.0286],
        [19.4926, 12.4154,     nan,  1.2958],
        [19.4926, 24.8309,     nan,  4.7615],
        ...,
        [-0.3017, -0.2589, -1.3486, -2.6918],
        [-0.2330, -0.3999, -2.6399, -0.5174],
        [-0.4660, -0.2000,  0.8258, -3.9831]])

In [38]:
(d[1]==13).nonzero()

tensor([], size=(0, 1), dtype=torch.int64)

In [14]:
%debug

> <ipython-input-11-a49969dbc8ff>(42)__getitem__()
     40             lables_t.append(i[1])
     41         w,h = x.size
---> 42         boxes_t /=torch.Tensor([w,h,w,h]).expand_as(boxes_t)
     43         lables_t = torch.LongTensor(label)
     44 

ipdb> exit


In [145]:
loc_target, conf_target = Data_encode.encode(boxes_t,lables_t)

## 3. Largest Item classifier

In [94]:
vgg = tv.models.vgg16_bn(pretrained = True)
print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [13]:
class SSD300(nt.NeuralNetwork):
    input_size = 300
    
    def __init__(self, num_classes, fine_tuning=False):
        super(SSD300, self).__init__()

        # model
        self.base = self.VGG16()
        self.norm4 = L2Norm2d(20)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1, dilation=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1, dilation=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1, dilation=1)

        self.conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)

        self.conv7 = nn.Conv2d(1024, 1024, kernel_size=1)

        self.conv8_1 = nn.Conv2d(1024, 256, kernel_size=1)
        self.conv8_2 = nn.Conv2d(256, 512, kernel_size=3, padding=1, stride=2)

        self.conv9_1 = nn.Conv2d(512, 128, kernel_size=1)
        self.conv9_2 = nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=2)

        self.conv10_1 = nn.Conv2d(256, 128, kernel_size=1)
        self.conv10_2 = nn.Conv2d(128, 256, kernel_size=3)

        self.conv11_1 = nn.Conv2d(256, 128, kernel_size=1)
        self.conv11_2 = nn.Conv2d(128, 256, kernel_size=3)

        # multibox layer
        self.multibox = MultiBoxLayer()
        self.multiboxloss = MultiBoxLoss()       

    def VGG16(self):
        '''VGG16 layers.'''
        cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.ReLU(True)]
                in_channels = x
        return nn.Sequential(*layers)  
    
    def forward(self, x):
        hs = []
        h = self.base(x)
        hs.append(self.norm4(h))  # conv4_3

        h = F.max_pool2d(h, kernel_size=2, stride=2, ceil_mode=True)

        h = F.relu(self.conv5_1(h))
        h = F.relu(self.conv5_2(h))
        h = F.relu(self.conv5_3(h))
        h = F.max_pool2d(h, kernel_size=3, padding=1, stride=1, ceil_mode=True)

        h = F.relu(self.conv6(h))
        h = F.relu(self.conv7(h))
        hs.append(h)  # conv7

        h = F.relu(self.conv8_1(h))
        h = F.relu(self.conv8_2(h))
        hs.append(h)  # conv8_2

        h = F.relu(self.conv9_1(h))
        h = F.relu(self.conv9_2(h))
        hs.append(h)  # conv9_2

        h = F.relu(self.conv10_1(h))
        h = F.relu(self.conv10_2(h))
        hs.append(h)  # conv10_2

        h = F.relu(self.conv11_1(h))
        h = F.relu(self.conv11_2(h))
        hs.append(h)  # conv11_2

        loc_preds, conf_preds = self.multibox(hs)
        return (loc_preds, conf_preds)
    
    def criterion(self, y, d):
        loc_preds,conf_preds = y
        loc_targets,conf_targets = d
        loss = self.multiboxloss(loc_preds, loc_targets, conf_preds, conf_targets)        
        return loss


In [14]:
net = SSD300()

TypeError: __init__() missing 1 required positional argument: 'num_classes'